# Day 4 - Giant Squid

>You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine.
>
>Maybe it wants to play bingo?
>
>Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)
>
>The submarine has a bingo subsystem to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:
>
```
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
```
>
> The score of the winning board can now be calculated. Start by finding the sum of all unmarked numbers on that board; in this case, the sum is 188. Then, multiply that sum by the number that was just called when the board won, 24, to get the final score, 188 * 24 = 4512.

Let's build a class!

Every bingo board should know their own state: numbers in the board, what's marked and what is the state of the game.

Whenever a new number is called, we `mark()` it to our board and at the same time, check if that number caused our board to win or not.

In [130]:
class BingoBoard:
    
    def __init__(self, board):
        self.board = board or []
        self.last_marked = None
        self.finished = False
        
    def from_text(text):
        text = text.strip()
        rows = text.split('\n')
        board = BingoBoard([[int(v) for v in row.split()] for row in rows])
        return board
        
    def mark(self, number):
        marked = False
        self.last_marked = number
        for row in self.board:
            if number in row:
                marked = True
                row[row.index(number)] = None
        if marked:
            return self.check_win()
        else:
            return False
            
    def check_win(self):
        # check rows
        for row in self.board:
            unfinished = any([value is not None for value in row])
            if not unfinished:
                self.finished = True
                return True
        
        # check columns
        for index in range(5):
            col = [self.board[i][index] for i, _ in enumerate(self.board)]
            unfinished = any([value is not None for value in col])
            if not unfinished:
                self.finished = True
                return True

        return False
    
    def calculate_unmarked_sum(self):
        sum = 0
        for row in self.board:
            for value in row:
                if value:
                    sum += value
        return sum
    
    def final_score(self):
        return self.calculate_unmarked_sum() * self.last_marked
    
    def __repr__(self):
        return '\n'.join('\t'.join(str(value) for value in row) for row in self.board)

## Part 1

In [142]:
from utils import read_bingo_input

numbers, boards = read_bingo_input()
numbers = [int(n) for n in numbers.split(',')]
bingo_boards = []
for board in boards:
    bingo_boards.append(BingoBoard.from_text(board))

def play_bingo(numbers, bingo_boards):
    for number in numbers:
        for board in bingo_boards:
            finished = board.mark(number)
            if finished:
                return board

final = play_bingo(numbers, bingo_boards)
print(final)
print(f'Last called: {final.last_marked}')
print(f'Score: {final.final_score()}')

assert final.final_score() == 11774

60	None	37	None	73
80	None	None	30	64
77	None	None	1	45
79	None	11	12	51
25	None	68	67	61
Last called: 14
Score: 11774


## Part 2

> On the other hand, it might be wise to try a different strategy: let the giant squid win.
>
>You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to figure out which board will win last and choose that one. That way, no matter which boards it picks, it will win for sure.
>
>In the above example, the second board is the last to win, which happens after 13 is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to 148 for a final score of 148 * 13 = 1924.

In [129]:
#Re-reading the inputs to avoid previous round affecting the results

numbers, boards = read_bingo_input()
numbers = [int(n) for n in numbers.split(',')]
bingo_boards = []
for board in boards:
    bingo_boards.append(BingoBoard.from_text(board))


def play_bingo_last(numbers, boards):
    winning_boards = []    
    for number in numbers:
        for board in boards:
            if not board.finished:
                finished = board.mark(number)
                if finished:
                    winning_boards.append(board)
    return winning_boards[-1]

result = play_bingo_last(numbers, bingo_boards)

print(result)
print(f'Solution: {result.final_score()}')
            


None	None	None	None	None
None	None	None	None	4
None	None	None	None	0
None	54	None	82	None
5	None	None	None	None
Solution: 4495
